In [2]:
import os
import sys
import glob

try:
    sys.path.append(glob.glob('carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import math
import time
import hydra
import logging
import argparse


from numpy import random
from omegaconf import DictConfig, OmegaConf, ValidationError



from core.pgm import PGM
from utils.weather import Weather
from core.vehicle_manager import VehicleManager
from core.pedestrian_manager import PedestrianManager
from utils.common import get_actor_blueprints
from shcemas.pgm_schema import PGMSchema

from hydra.core.config_store import ConfigStore
from shcemas.pgm_schema import PGMModel

In [2]:



cs = ConfigStore.instance()
cs.store(name="base_config", node=PGMSchema)
cs.store(group="model", name="base_model", node=PGMModel)
with hydra.initialize(version_base=None, config_path="conf/pgm"):
    cfg: PGMSchema = hydra.compose(config_name="config")
    # print(cfg)
    # print(OmegaConf.to_yaml(cfg.model))
    pgm = PGM(cfg)
    pgm.print_cpd('T')

+----------+------------+---------------+---------------+
| R        | R(NO_RAIN) | R(LIGHT_RAIN) | R(HEAVY_RAIN) |
+----------+------------+---------------+---------------+
| T(LOW)   | 0.1        | 0.4           | 0.9           |
+----------+------------+---------------+---------------+
| T(HEAVY) | 0.9        | 0.6           | 0.1           |
+----------+------------+---------------+---------------+


In [3]:
states = pgm.get_states()
states.Rain.NO_RAIN.name

'NO_RAIN'

In [5]:
variables = pgm.get_variables()
variables.Rain

'R'

In [6]:
res = pgm.predict_dist([variables.Traffic, variables.Speed], evidence={variables.Rain: states.Rain.LIGHT_RAIN.name})
print(res)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+----------+---------+------------+
| T        | S       |   phi(T,S) |
+==========+=========+============+
| T(LOW)   | S(LOW)  |     0.2400 |
+----------+---------+------------+
| T(LOW)   | S(HIGH) |     0.1600 |
+----------+---------+------------+
| T(HEAVY) | S(LOW)  |     0.3600 |
+----------+---------+------------+
| T(HEAVY) | S(HIGH) |     0.2400 |
+----------+---------+------------+


In [33]:
res = pgm.predict_state(['T', 'S'], evidence={'R': states.Rain.LIGHT_RAIN.name})
res

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


{'T': 'HEAVY', 'S': 'LOW'}

In [25]:
def main(cfg: DictConfig):
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
    
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)

    cs = ConfigStore.instance()
    cs.store(name="base_pgm_config", node=PGMSchema)
    cs.store(group="model", name="base_pgm_model", node=PGMModel)
    pgm_cfg: PGMSchema = hydra.compose(config_name="pgm_config.yaml")
    # print(pgm_cfg)
    pgm = PGM(pgm_cfg)
    pgm_variables = pgm.get_variables()
    pgm_states = pgm.get_states()
    traffic_state = pgm.predict_state(
        [pgm_variables.Traffic], 
        evidence={
            pgm_variables.Rain: pgm_states.Rain.HEAVY_RAIN.name
        }
    )[pgm_variables.Traffic]

    if traffic_state == pgm_states.Traffic.LOW.name:
        num_of_vehicles = 10
    else:
        num_of_vehicles = 30

    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    # print(vcfg)
    tm = VehicleManager(client=client, cfg=vcfg)

    pcfg = hydra.compose(config_name="pedestrian_manager.yaml")
    # print(pcfg)
    pm = PedestrianManager(client=client, cfg=pcfg)
    try:
        world = client.get_world()

        tm.spawn_vehicles(number_of_vehicles=num_of_vehicles)
        pm.spawn_walkers(number_of_walkers=pcfg.number_of_walkers)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor

        weather = Weather(world.get_weather())

        elapsed_time = 0.0

        synchronous_master = world.get_settings().synchronous_mode
        while True:
            if not cfg.asynch and synchronous_master:
                world.tick()
            else:
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
                if elapsed_time > update_freq:
                    weather.tick(speed_factor * elapsed_time)

                    world.set_weather(weather.weather)
                    sys.stdout.write('\r' + str(weather) + 12 * ' ')
                    sys.stdout.flush()
                    elapsed_time = 0.0

    finally:

        tm.destroy()
        pm.destroy()
        time.sleep(0.5)

In [26]:
try:
    with hydra.initialize(version_base=None, config_path="conf"):
        cfg = hydra.compose(config_name="config.yaml")
        # print(cfg)
        main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


You are currently in asynchronous mode. If this is a traffic simulation,             you could experience some issues. If it's not working correctly, switch to synchronous             mode by using self.traffic_manager.set_synchronous_mode(True)


ERROR: Spawn failed because of collision at spawn position
ERROR: Spawn failed because of collision at spawn position
ERROR: Spawn failed because of collision at spawn position



Added 10 vehicles
Spawned 7 walkers.
Sun(alt: 28.52, azm: 332.92) Storm(clouds=90%, rain=74%, wind=90%)            
Destroying 10 vehicles
Destroying 7 walkers

done.
